In [ ]:
import cv2
from ultralytics import YOLO
import pandas as pd
from datetime import datetime, timedelta

def process_video(video_path, output_csv):
    # YOLOv8モデルの読み込み（人などの物体検出用）
    model = YOLO('yolov8m.pt')  # 精度重視なら 'yolov8x.pt', 速度重視なら 'yolov8n.pt'

    # 動画の読み込み
    cap = cv2.VideoCapture(video_path)
    
    # ログ保存用リスト
    log_data = []
    
    # 動画の開始時刻（仮定：ファイル名やメタデータから取得する想定）
    start_time = datetime.strptime("2025-04-10 17:40:27", "%Y-%m-%d %H:%M:%S")
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    frame_count = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # フレームごとに推論を実行（トラッキング有効化）
        # persist=True で前のフレームのIDを引き継ぐ
        results = model.track(frame, persist=True, classes=[0], verbose=False) # class 0 is 'person'

        # 現在の時刻計算
        current_time = start_time + timedelta(seconds=frame_count / frame_rate)
        timestamp_str = current_time.strftime("%H:%M:%S")

        # 検出された人数
        num_people = 0
        people_ids = []
        
        # 検出結果の解析
        if results[0].boxes.id is not None:
            boxes = results[0].boxes
            track_ids = boxes.id.int().cpu().tolist()
            # confidences = boxes.conf.float().cpu().tolist()
            
            num_people = len(track_ids)
            people_ids = track_ids
            
            # ここで各人の座標(x,y)により「レジ前」「席」などのエリア判定が可能
            # 例: for box, track_id in zip(boxes.xyxy, track_ids): ...

        # 1秒ごとのデータをログに残す（フレームレートで割った余りが0の時など）
        # ここでは簡易的に毎フレームのデータを間引いて処理する想定で全件追加または調整
        if frame_count % int(frame_rate) == 0:  # 1秒に1回ログ出力
            print(f"Time: {timestamp_str} | Count: {num_people} | IDs: {people_ids}")
            log_data.append({
                "Timestamp": timestamp_str,
                "Person_Count": num_people,
                "Person_IDs": people_ids
            })

        # （オプション）画面にバウンディングボックスを描画して表示
        # annotated_frame = results[0].plot()
        # cv2.imshow("YOLOv8 Analysis", annotated_frame)
        # if cv2.waitKey(1) & 0xFF == ord("q"):
        #     break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

    # CSVに保存
    df = pd.DataFrame(log_data)
    df.to_csv(output_csv, index=False)
    print(f"Analysis saved to {output_csv}")

# 実行
# video_pathには実際の動画ファイルまたは連番画像のパスを指定
# process_video('input_video.mp4', 'output_log.csv')